# DigitalTWINS on FHIR - Live demo

## Introduction
The examples considered in this demo involve:
- 2 patients for Exemplar Project 1 – Biomarkers for pulmonary hypertension (note that a simplified workflow with one step/tool is used here).
- 2 patients for Exemplar Project 4 - Breast cancer reporting (6 step workflow).
- 1 patient contributed to both projects (ie in this scenario, they have both pulmonary hypertension and breast cancer).

## Definitions
- FHIR - Fast Healthcare Interoperability Resources
- FHIR Server - a server implemented according to the FHIR standard, allow users to create, update, delete, and search FHIR health data.
- Primary Measurements - a SPARC SDS dataset.
- CWL - Common Workflow Language.
- Workflow tool process - generated by executing a tool/step of a workflow.

## Learning outcomes

- Learn how to install the digitaltwins-on-fhir python client
- Learn how to find all primary measurements for a patient that have been contributed from multiple research studies. This includes:
  -  finding FHIR ImagingStudy resources and their PACS endpoints.
- Learn how to find which workflow and tool generated a specific derived measurement observation.
- Learn how to find all tools and models used by a workflow and their workflow tool processes.
- Learn how to find inputs and outputs of a given tool in a workflow.




## Installing the digitaltwins-on-fhir python client

Install package


*   pip install digitaltwins-on-fhir

In [ ]:
pip install digitaltwins-on-fhir

In [ ]:
from digitaltwins_on_fhir import Adapter
from pprint import pprint
adapter = Adapter("http://localhost:8080/fhir")
search = adapter.search()

## Example 1: Finding all primary measurements for a patient

Let's randomly select a patient

Step 1: Find the random patient's uuid and get all dataset composition resources.

In [ ]:
measurements = await search.get_patient_measurements("c6923eb4-a5c2-4239-8b7a-16d1268b108d")
pprint(measurements)


# Example 2: Find which workflow, tool, and primary data was used to generate a specific derived measurement observation

It might of interest to find out more information regarding the provenance of a give observation e.g. called "tumour position" (uuid: `231d9946-949a-4fee-8695-5887209bd2db_2673e5a3-8437-41f5-9fef-0983f5662e93_Workflow-Process-Output-Observation-0-0`). For example, we could be interested in finding:
- which assay, workflow and tool generated this observation.

We can start by defining the observation of interest:

In [ ]:
res = await search.get_workflow_details_by_derived_data("Observation", "231d9946-949a-4fee-8695-5887209bd2db_2673e5a3-8437-41f5-9fef-0983f5662e93_Workflow-Process-Output-Observation-0-0")
pprint(res)

We can find all inputs and their dataset uuid for generating the Observation

In [ ]:
res = await search.get_all_inputs_by_derived_data("Observation", "231d9946-949a-4fee-8695-5887209bd2db_2673e5a3-8437-41f5-9fef-0983f5662e93_Workflow-Process-Output-Observation-0-0")
pprint(res)

## Example 3: Find all tools and models used by a workflow and their workflow tool processes

Lets find all workflows, and then choose a specific uuid that we can use as an example for finding all tools used by the workflow.

We can also find a workflow by searching by its `name` e.g. "Automated torso model generation - script" direactly.

We can then find all the tools that were used in this workflow including the software and/or models used in the tool (here we show how we can access the workflow from its UUID).

We can also find all the workflow tool process that have been run for that particular workflow.

In [ ]:
res = await search.get_all_workflow_tools_by_workflow(name="Automated torso model generation - script")
pprint(res)

## Example 4 Find inputs and outputs of a given tool in a workflow
Here we will specify some of the resources that we are looking for, however, all this information can be queried as shown in the previous examples.:

*   workflow uuid: 4c36c076-3813-4247-8317-e163901b1ae3
*   workflow tool uuid: 2673e5a3-8437-41f5-9fef-0983f5662e93

By specifying the workflow and workflow tool UUIDs above, we can retrieve all related assays and workflow tool processes.


Now we have all the processes for that workflow tool.

We can then find the input and output of each workflow tool process:

In [ ]:
res = await search.get_all_inputs_outputs_of_workflow_tool(name="Tumour Position Correction (Manual) Tool")
pprint(res)